In [13]:
from scipy.fftpack import fft, ifft
import scipy.io.wavfile as wav
import scipy.signal as sg
import numpy as np


In [113]:
import numpy as np
import soundfile as sf
import librosa

def spectral_subtraction(audio, reduction_factor=1.0):
    # Compute the power spectrum of the audio signal
    audio_spec = np.abs(librosa.stft(audio))

    # Estimate the noise spectrum from a portion of the audio
    noise_frames = audio_spec[:, 188:]  # Adjust the number of frames as needed
    avg_noise_spec = np.mean(noise_frames, axis=1, keepdims=True)

    # Compute the noise reduction factor
    reduction_factor = np.maximum(0, reduction_factor)

    # Apply spectral subtraction
    enhanced_spec = audio_spec - (reduction_factor * avg_noise_spec)

    # Inverse STFT to obtain the enhanced audio signal
    enhanced_audio = librosa.istft(enhanced_spec)

    return enhanced_audio

# Example usage
# Load the audio signal
audio, sr = librosa.load('/media/ahmed/DATA/forth year/Graduation Project/tacron2/tts/6.wav', sr=None)

# Apply spectral subtraction with a reduction factor of 0.5
reduced_noise_audio = spectral_subtraction(audio, reduction_factor=0.7)

# Save the enhanced audio signal to a file
sf.write('/media/ahmed/DATA/forth year/Graduation Project/tacron2/tts/ot.wav', reduced_noise_audio, sr)


In [62]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.io import wavfile

def apply_transfer(signal, transfer, interpolation='linear'):
    constant = np.linspace(-1, 1, len(transfer))
    interpolator = interp1d(constant, transfer, interpolation)
    return interpolator(signal)

In [63]:
# hard limiting
def limiter(x, treshold=0.8):
    transfer_len = 1000
    transfer = np.concatenate([ np.repeat(-1, int(((1-treshold)/2)*transfer_len)),
                                np.linspace(-1, 1, int(treshold*transfer_len)),
                                np.repeat(1, int(((1-treshold)/2)*transfer_len)) ])
    return apply_transfer(x, transfer)

# smooth compression: if factor is small, its near linear, the bigger it is the
# stronger the compression
def arctan_compressor(x, factor=2):
    constant = np.linspace(-1, 1, 1000)
    transfer = np.arctan(factor * constant)
    transfer /= np.abs(transfer).max()
    return apply_transfer(x, transfer)

In [64]:
sr, x = wavfile.read("ot.wav")
x = x / np.abs(x).max() # x scale between -1 and 1

x2 = limiter(x)
x2 = np.int16(x2 * 32767)
wavfile.write("output_limit.wav", sr, x2)

x3 = arctan_compressor(x)
x3 = np.int16(x3 * 32767)
wavfile.write("output_comp.wav", sr, x3)